# 📊 Évaluation RAG Manuel avec Ragas - Projet Néo

## 🎯 Objectif
Évaluer la qualité d'un système RAG en utilisant les 5 métriques Ragas principales :
1. **Faithfulness** : Fidélité aux sources (pas d'hallucination)
2. **Answer Correctness** : Correction vs référence métier
3. **Answer Relevancy** : Pertinence à la question
4. **Context Precision** : Qualité du ranking des contextes
5. **Context Recall** : Complétude de la récupération

## 📁 Données utilisées
- Fichier CSV : `reference_qa_manuel_template.csv`
- 12 questions du Projet Néo avec réponses de référence
- Contextes formatés avec séparateur `|||`

## 0) Installation des dépendances

In [1]:
# Exécutez cette cellule si besoin (versions recommandées).
# Si vous êtes hors ligne, installez ces paquets au préalable dans votre environnement.
# !pip install "ragas>=0.3.1,<0.4" "langchain>=0.2" "datasets>=2.20" pandas tiktoken

import sys, platform, importlib

def check_pkg(name):
    try:
        m = importlib.import_module(name)
        return m.__version__ if hasattr(m, "__version__") else "installed"
    except Exception as e:
        return f"not found: {e}"

print("Python:", sys.version)
print("OS:", platform.platform())
print("ragas:", check_pkg("ragas"))
print("langchain:", check_pkg("langchain"))
print("datasets:", check_pkg("datasets"))
print("pandas:", check_pkg("pandas"))

Python: 3.13.5 | packaged by conda-forge | (main, Jun 16 2025, 08:27:50) [GCC 13.3.0]
OS: Linux-5.15.153.1-microsoft-standard-WSL2-x86_64-with-glibc2.39


/opt/conda/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ragas: 0.3.2
langchain: 0.3.27
datasets: 4.0.0
pandas: 2.3.2


## 1️⃣ Configuration

In [2]:
import os

# === Choix du fournisseur LLM ===
# Options: "openai", "claude", "gemini", "ollama"
RAGAS_LLM_PROVIDER = os.getenv("RAGAS_LLM_PROVIDER", "openai").lower()

# Modèles par défaut (changez si besoin)
OPENAI_MODEL  = os.getenv("OPENAI_MODEL",  "gpt-4o-mini")  # ou "gpt-4o"
CLAUDE_MODEL  = os.getenv("CLAUDE_MODEL",  "claude-3-5-sonnet-20240620")
GEMINI_MODEL  = os.getenv("GEMINI_MODEL",  "gemini-1.5-pro")
OLLAMA_MODEL  = os.getenv("OLLAMA_MODEL",  "llama3.1:8b")

# Clés d'API attendues dans l'environnement (ne les mettez pas en dur dans le notebook)
os.environ["OPENAI_API_KEY"]     = ""
# os.environ["ANTHROPIC_API_KEY"]  = "..."
# os.environ["GOOGLE_API_KEY"]     = "..."

# === Données ===
# Si vous exécutez ce notebook localement, placez votre CSV dans le même dossier que le notebook
# ou indiquez le chemin absolu complet ci-dessous.
DATA_PATH = os.getenv("DATA_PATH", "reference_qa_manuel_template.csv")

# Dossiers de sortie
OUTPUT_DIR = os.getenv("OUTPUT_DIR", "outputs")
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("Provider:", RAGAS_LLM_PROVIDER)
print("Data path:", DATA_PATH)
print("Output dir:", OUTPUT_DIR)

Provider: openai
Data path: reference_qa_manuel_template.csv
Output dir: outputs


## 2️⃣ Chargement du CSV et aperçu des données

In [3]:
import pandas as pd

if not os.path.exists(DATA_PATH):
    
    alt_path = '../data/reference/reference_qa_manuel_template.csv'
    if os.path.exists(alt_path):
        DATA_PATH = alt_path
        print(f"INFO: DATA_PATH non trouvé, utilisation de {alt_path}")
    else:
        raise FileNotFoundError(f"CSV introuvable: {DATA_PATH}")

raw_df = pd.read_csv(DATA_PATH)
print("Shape:", raw_df.shape)
display(raw_df.head(5))
print("Colonnes:", list(raw_df.columns))

INFO: DATA_PATH non trouvé, utilisation de ../data/reference/reference_qa_manuel_template.csv
Shape: (12, 8)


,question_id,question,reference_answer,sharepoint_document,ragas_question,ragas_answer,ragas_contexts,ragas_ground_truth
0,QNeo001,Qui est le chef de projet du Projet Néo ?,Marc Dubois est le chef de projet du Projet Néo.,Note de cadrage - Projet Neo.txt,Qui est le chef de projet du Projet Néo ?,Marc Dubois est le chef de projet du Projet Né...,[Document: Note de cadrage - Projet Neo.txt] L...,Note de cadrage - Projet Neo.txt
1,QNeo002,Quel est l'objectif principal du Projet Néo ?,Le Projet Néo vise à développer un nouvel algo...,Note de cadrage - Projet Neo.txt,Quel est l'objectif principal du Projet Néo ?,L'objectif du Projet Néo est de développer un ...,[Document: Note de cadrage - Projet Neo.txt] L...,Note de cadrage - Projet Neo.txt
2,QNeo003,Qui est le lead developer assigné au projet ?,Sophie Martin est le lead developer assigné au...,Note de cadrage - Projet Neo.txt,Qui est le lead developer assigné au projet ?,Sophie Martin est désignée comme Lead Develope...,[Document: Note de cadrage - Projet Neo.txt] L...,Note de cadrage - Projet Neo.txt
3,QNeo004,Quelles sont les compétences de David Chen ?,"David Chen possède des compétences en Python, ...",Repertoire equipe - Projet Neo.txt,Quelles sont les compétences de David Chen ?,"David Chen, Data Scientist Principal, maîtrise...",[Document: Repertoire equipe - Projet Neo.txt]...,Repertoire equipe - Projet Neo.txt
4,QNeo005,Qui est le manager de Sophie Martin ?,Marc Dubois est le manager de Sophie Martin.,Repertoire equipe - Projet Neo.txt,Qui est le manager de Sophie Martin ?,Marc Dubois est le manager de Sophie Martin se...,[Document: Repertoire equipe - Projet Neo.txt]...,Repertoire equipe - Projet Neo.txt


Colonnes: ['question_id', 'question', 'reference_answer', 'sharepoint_document', 'ragas_question', 'ragas_answer', 'ragas_contexts', 'ragas_ground_truth']


## 3️⃣ Normalisation du dataset pour RAGAS

In [4]:

import ast
import math

df = raw_df.copy()

# === Mapping des colonnes présentes dans votre CSV ===
# Votre CSV inclut (exemple): question, ragas_answer, ragas_contexts, reference_answer
# On mappe vers les colonnes attendues par RAGAS : question, answer, contexts (List[str]), ground_truth (str)
QUESTION_COL   = "question"          # ou "ragas_question"
ANSWER_COL     = "ragas_answer"      # réponse modèle
CONTEXTS_COL   = "ragas_contexts"    # passages récupérés (un ou plusieurs)
GROUNDTRUTH_COL_IN = "reference_answer"  # vérité terrain métier (texte)

if QUESTION_COL not in df.columns:
    # fallback si besoin
    if "ragas_question" in df.columns:
        QUESTION_COL = "ragas_question"
    else:
        raise KeyError("Aucune colonne question détectée. Attendu 'question' ou 'ragas_question'.")

if ANSWER_COL not in df.columns:
    # fallback si besoin
    if "answer" in df.columns:
        ANSWER_COL = "answer"
    else:
        raise KeyError("Aucune colonne answer détectée. Attendu 'ragas_answer' ou 'answer'.")

if CONTEXTS_COL not in df.columns:
    if "contexts" in df.columns:
        CONTEXTS_COL = "contexts"
    else:
        raise KeyError("Aucune colonne contexts détectée. Attendu 'ragas_contexts' ou 'contexts'.")

if GROUNDTRUTH_COL_IN not in df.columns:
    # fallback si besoin
    if "reference" in df.columns:
        GROUNDTRUTH_COL_IN = "reference"
    elif "ground_truth" in df.columns:
        GROUNDTRUTH_COL_IN = "ground_truth"
    else:
        raise KeyError("Aucune ground truth détectée. Attendu 'reference_answer' ou 'reference' ou 'ground_truth'.")

def to_list_contexts(x):
    """Convertit la colonne contexts en List[str].
    - Si déjà une liste (str formatée comme '["a","b"]'), on la parse avec ast.literal_eval
    - Si séparateur '|||' ou ';' -> on split
    - Sinon -> liste à un élément
    """
    if x is None or (isinstance(x, float) and math.isnan(x)):
        return []
    if isinstance(x, list):
        return [str(xx) for xx in x]
    if isinstance(x, str):
        s = x.strip()
        # Cas liste déjà sérialisée
        if (s.startswith("[") and s.endswith("]")) or (s.startswith("(") and s.endswith(")")):
            try:
                parsed = ast.literal_eval(s)
                if isinstance(parsed, (list, tuple)):
                    return [str(xx) for xx in parsed]
            except Exception:
                pass
        # Séparateurs courants
        for sep in ["|||", "§§", ";;", "##", "\n"]:
            if sep in s:
                return [ss.strip() for ss in s.split(sep) if ss.strip()]
        # Fallback: un seul contexte
        return [s]
    # Autres types -> string
    return [str(x)]

dataset_dict = {
    "question":     df[QUESTION_COL].astype(str).tolist(),
    "answer":       df[ANSWER_COL].astype(str).tolist(),
    "contexts":     [to_list_contexts(v) for v in df[CONTEXTS_COL].tolist()],
    "ground_truth": df[GROUNDTRUTH_COL_IN].astype(str).tolist(),  # mapping important pour answer_correctness
}

# Sanity check rapide
for k, v in dataset_dict.items():
    print(k, ":", type(v), f"(len={len(v)})")
print("Exemple contexts[0]:", dataset_dict["contexts"][0][:2] if dataset_dict["contexts"] else "n/a")


question : <class 'list'> (len=12)
answer : <class 'list'> (len=12)
contexts : <class 'list'> (len=12)
ground_truth : <class 'list'> (len=12)
Exemple contexts[0]: ['[Document: Note de cadrage - Projet Neo.txt] Le Projet Néo vise à développer un nouvel algorithme de recommandation pour notre plateforme e-commerce.', "[Document: Note de cadrage - Projet Neo.txt] L'équipe clé comprend Marc Dubois comme Chef de Projet, Sophie Martin comme Lead Developer et David Chen comme Expert Data Science."]


In [5]:
## Construction du Dataset 
from datasets import Dataset as HFDataset

hf_dataset = HFDataset.from_dict(dataset_dict)
hf_dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth'],
    num_rows: 12
})

## 4️⃣ Les 5 Métriques Ragas Expliquées

### 1. 🎯 **Faithfulness** (Fidélité) - Score 0-1
- **Mesure** : La réponse est-elle factuellement cohérente avec les contextes ?
- **Données** : `contexts` + `answer`
- **Objectif** : Détecter les hallucinations

### 2. ✅ **Answer Correctness** (Correction) - Score 0-1
- **Mesure** : La réponse est-elle correcte vs référence métier ?
- **Données** : `answer` + `reference`
- **Objectif** : Valider la conformité métier

### 3. 💬 **Answer Relevancy** (Pertinence) - Score 0-1
- **Mesure** : La réponse répond-elle à la question ?
- **Données** : `question` + `answer`
- **Objectif** : Éviter les réponses hors-sujet

### 4. 🎯 **Context Precision** (Précision) - Score 0-1
- **Mesure** : Les contextes pertinents sont-ils bien classés ?
- **Données** : `question` + `contexts` + `ground_truths`
- **Objectif** : Évaluer le ranking

### 5. 📚 **Context Recall** (Rappel) - Score 0-1
- **Mesure** : Tous les contextes importants récupérés ?
- **Données** : `contexts` + `ground_truths`
- **Objectif** : Évaluer la complétude

## 5️⃣ Préparation du Dataset Ragas

In [6]:
from ragas.llms import LangchainLLMWrapper

def build_llm(provider: str):
    provider = provider.lower().strip()
    if provider == "openai":
        from langchain_openai import ChatOpenAI
        lc = ChatOpenAI(model=OPENAI_MODEL, temperature=0)
        return LangchainLLMWrapper(lc)
    elif provider == "claude":
        from langchain_anthropic import ChatAnthropic
        lc = ChatAnthropic(model=CLAUDE_MODEL, temperature=0)
        return LangchainLLMWrapper(lc)
    elif provider == "gemini":
        from langchain_google_genai import ChatGoogleGenerativeAI
        lc = ChatGoogleGenerativeAI(model=GEMINI_MODEL, temperature=0)
        return LangchainLLMWrapper(lc)
    elif provider == "ollama":
        # Selon la version de LangChain : ChatOllama (chat) ou Ollama (llm)
        try:
            from langchain_community.chat_models import ChatOllama
            lc = ChatOllama(model=OLLAMA_MODEL)
        except Exception:
            from langchain_community.llms import Ollama
            lc = Ollama(model=OLLAMA_MODEL)
        return LangchainLLMWrapper(lc)
    else:
        raise ValueError(f"Provider non supporté: {provider}")

llm = build_llm(RAGAS_LLM_PROVIDER)
print("✅ LLM prêt pour RAGAS:", type(llm).__name__, "| provider:", RAGAS_LLM_PROVIDER)




✅ LLM prêt pour RAGAS: LangchainLLMWrapper | provider: openai


## 6️⃣ Définition des métriques ragas

In [7]:
from ragas.metrics import (
    faithfulness,
    answer_correctness,
    answer_relevancy,   # ← au lieu de response_relevancy
    context_precision,
    context_recall,
)

metrics = [
    faithfulness,
    answer_correctness,
    answer_relevancy,   # ← idem ici
    context_precision,
    context_recall,
]

metrics

[Faithfulness(_required_columns={<MetricType.SINGLE_TURN: 'single_turn'>: {'response', 'user_input', 'retrieved_contexts'}}, name='faithfulness', llm=None, output_type=<MetricOutputType.CONTINUOUS: 'continuous'>, nli_statements_prompt=NLIStatementPrompt(instruction=Your task is to judge the faithfulness of a series of statements based on a given context. For each statement you must return verdict as 1 if the statement can be directly inferred based on the context or 0 if the statement can not be directly inferred based on the context., examples=[(NLIStatementInput(context='John is a student at XYZ University. He is pursuing a degree in Computer Science. He is enrolled in several courses this semester, including Data Structures, Algorithms, and Database Management. John is a diligent student and spends a significant amount of time studying and completing assignments. He often stays late in the library to work on his projects.', statements=['John is majoring in Biology.', 'John is taking

## 7️⃣ Évaluation avec Ragas

In [8]:

from ragas import evaluate

# evaluate() accepte un HF Dataset + métriques + LLM
# Pas besoin de column_map si vos clés s'appellent déjà: question, answer, contexts, ground_truth
# (Nous avons mappé au préalable 'reference_answer' -> 'ground_truth')

result = evaluate(
    dataset=hf_dataset,
    metrics=metrics,
    llm=llm,
    raise_exceptions=False,
    show_progress=True,
)

print("✅ Évaluation terminée.")
df_results = result.to_pandas()
display(df_results.head(10))

# Sauvegarde brute
csv_out = os.path.join(OUTPUT_DIR, "ragas_raw_results.csv")
df_results.to_csv(csv_out, index=False, encoding="utf-8")
print("Résultats enregistrés ->", csv_out)


Evaluating: 100%|██████████| 60/60 [00:16<00:00,  3.66it/s]


✅ Évaluation terminée.


,user_input,retrieved_contexts,response,reference,faithfulness,answer_correctness,answer_relevancy,context_precision,context_recall
0,Qui est le chef de projet du Projet Néo ?,[[Document: Note de cadrage - Projet Neo.txt] ...,Marc Dubois est le chef de projet du Projet Né...,Marc Dubois est le chef de projet du Projet Néo.,1.000000,0.743916,0.998186,0.5,1.0
1,Quel est l'objectif principal du Projet Néo ?,[[Document: Note de cadrage - Projet Neo.txt] ...,L'objectif du Projet Néo est de développer un ...,Le Projet Néo vise à développer un nouvel algo...,1.000000,0.993853,0.985759,1.0,1.0
2,Qui est le lead developer assigné au projet ?,[[Document: Note de cadrage - Projet Neo.txt] ...,Sophie Martin est désignée comme Lead Develope...,Sophie Martin est le lead developer assigné au...,1.000000,0.996659,0.930813,1.0,1.0
3,Quelles sont les compétences de David Chen ?,[[Document: Repertoire equipe - Projet Neo.txt...,"David Chen, Data Scientist Principal, maîtrise...","David Chen possède des compétences en Python, ...",1.000000,0.907657,0.881353,1.0,1.0
4,Qui est le manager de Sophie Martin ?,[[Document: Repertoire equipe - Projet Neo.txt...,Marc Dubois est le manager de Sophie Martin se...,Marc Dubois est le manager de Sophie Martin.,0.500000,0.745691,0.970598,1.0,1.0
5,Quelle est la date limite pour le premier PoC ...,[[Document: Compte-Rendu de Reunion.txt] Proch...,Sophie Martin doit livrer un premier PoC (Proo...,Sophie Martin doit livrer un premier PoC (Proo...,0.666667,0.846984,0.932647,1.0,1.0
6,Quel framework a été confirmé pour le modèle p...,[[Document: Compte-Rendu de Reunion.txt] Décis...,TensorFlow a été confirmé comme framework pour...,TensorFlow a été confirmé comme framework pour...,1.000000,0.741270,0.967857,1.0,1.0
7,Qui dirige l'équipe Data Warehouse ?,[[Document: Repertoire equipe - Projet Neo.txt...,Hélène Petit occupe le poste de Directrice Dat...,Hélène Petit dirige l'équipe Data Warehouse.,1.000000,0.621241,0.765512,1.0,1.0
8,Quelle base de données est envisagée pour le p...,[[Document: Note de cadrage - Projet Neo.txt] ...,Neo4j est la base de données envisagée pour le...,Neo4j est envisagé comme base de données pour ...,1.000000,0.995710,0.882434,1.0,1.0
9,Quand est prévue la revue de projet avec Carol...,[[Document: Compte-Rendu de Reunion.txt] Proch...,La revue de projet avec Carole Lambert est pro...,Une revue de projet est fixée avec Carole Lamb...,1.000000,0.996939,0.990623,0.5,1.0


Résultats enregistrés -> outputs/ragas_raw_results.csv


## 8️⃣ Analyse des Résultats

In [9]:
import numpy as np
import json
from datetime import datetime
import os

# Détection du bon nom de colonne pour la pertinence
rel_col = "response_relevancy" if "response_relevancy" in df_results.columns else (
    "answer_relevancy" if "answer_relevancy" in df_results.columns else None
)

wanted_cols = ["faithfulness", "answer_correctness", "context_precision", "context_recall"]
if rel_col:
    wanted_cols.insert(2, rel_col)

present = [c for c in wanted_cols if c in df_results.columns]
summary = {c: float(np.nanmean(df_results[c])) for c in present}

print("📊 Scores moyens (0–1):")
for k, v in summary.items():
    print(f" - {k}: {v:.3f}")

summary_out = os.path.join(OUTPUT_DIR, "ragas_summary.json")
with open(summary_out, "w", encoding="utf-8") as f:
    json.dump({
        "generated_at": datetime.now().isoformat(),
        "provider": RAGAS_LLM_PROVIDER,
        "model": {
            "openai": OPENAI_MODEL,
            "claude": CLAUDE_MODEL,
            "gemini": GEMINI_MODEL,
            "ollama": OLLAMA_MODEL,
        }.get(RAGAS_LLM_PROVIDER, "n/a"),
        "scores": summary,
    }, f, ensure_ascii=False, indent=2)

print("Synthèse enregistrée ->", summary_out)


📊 Scores moyens (0–1):
 - faithfulness: 0.847
 - answer_correctness: 0.829
 - answer_relevancy: 0.929
 - context_precision: 0.861
 - context_recall: 1.000
Synthèse enregistrée -> outputs/ragas_summary.json
